In [ ]:
import warnings
from sklearn.metrics.pairwise import cosine_distances
import plotly.express as px
import gc
from datetime import datetime
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD




warnings.filterwarnings('ignore')

In [36]:
import nltk
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [37]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/clinicaltrials-gov-clinical-trials-dataset/clin_trials.csv


In [38]:
'''n = 496615 #number of records in file
s = 100000 #desired sample size

n = sum(1 for line in open('/kaggle/input/clinicaltrials-gov-clinical-trials-dataset/clin_trials.csv')) - 1 #number of records in file (excludes header)
s = 10000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list'''
clin_trials = pd.read_csv('/kaggle/input/clinicaltrials-gov-clinical-trials-dataset/clin_trials.csv')
clin_trials = clin_trials.drop('Unnamed: 0', axis = 1)
clin_trials_ro = clin_trials
clin_trials.head(1)

,Organization Full Name,Organization Class,Responsible Party,Brief Title,Full Title,Overall Status,Start Date,Standard Age,Conditions,Primary Purpose,Interventions,Intervention Description,Study Type,Phases,Outcome Measure,Medical Subject Headings
0,Montefiore Medical Center,OTHER,SPONSOR,Kinesiotape for Edema After Bilateral Total Kn...,"Effect of Kinesiotaping on Edema Management, P...",COMPLETED,2021-10-18,ADULT OLDER_ADULT,"Arthroplasty Complications, Arthroplasty, Repl...",TREATMENT,Kinesio(R)Tape for edema control,"Kinesio(R)Tape is an elastic, cotton tape with...",INTERVENTIONAL,NaN,Change from baseline and during 1-2-day time i...,Edema


In [39]:
pd.set_option('display.max_columns', None)
print(len(clin_trials))
clin_trials = clin_trials[clin_trials['Responsible Party'] == 'SPONSOR']
clin_trials = clin_trials[clin_trials['Organization Class'] == 'INDUSTRY']
clin_trials = clin_trials.reset_index(drop = True)
print(len(clin_trials))
clin_trials.head(1)

496615
100864


,Organization Full Name,Organization Class,Responsible Party,Brief Title,Full Title,Overall Status,Start Date,Standard Age,Conditions,Primary Purpose,Interventions,Intervention Description,Study Type,Phases,Outcome Measure,Medical Subject Headings
0,TRB Chemedica International SA,INDUSTRY,SPONSOR,Efficacy and Safety of BUFY01 Versus SVS20 in ...,Efficacy and Safety of BUFY01 Versus SVS20 in ...,NOT_YET_RECRUITING,2024-03,ADULT OLDER_ADULT,Dry Eye Disease,TREATMENT,"BUFY01 eye drops in single-dose containers, SV...","1 to 2 drops in each eye, as often as needed, ...",INTERVENTIONAL,NaN,Signs,Dry Eye Syndromes Keratoconjunctivitis Sicca E...


In [40]:
formats = [
    "%Y-%m-%d",             # 2023-06-01
    "%d/%m/%Y %H:%M:%S",    # 01/06/2023 14:45:00
    "%B %d, %Y",            # June 1, 2023
    "%Y.%m.%d AD at %H:%M:%S", # 2023.06.01 AD at 14:45:00
    "%d-%b-%Y",             # 01-Jun-2023
    "%Y/%m/%d %H:%M",       # 2023/06/01 14:45
    "%d/%m/%Y"              # 15/08/2023
]

In [41]:
def parse_mixed_date(date_str):
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    return pd.NaT

In [42]:
clin_trials.describe()

,Organization Full Name,Organization Class,Responsible Party,Brief Title,Full Title,Overall Status,Start Date,Standard Age,Conditions,Primary Purpose,Interventions,Intervention Description,Study Type,Phases,Outcome Measure,Medical Subject Headings
count,100864,100864,100864,100864,100864,100864,100864,100864,100864,100864,100864,100864,100864,84897,100864,100864
unique,11732,1,1,100346,98334,13,6635,6,32146,10,78488,79843,3,8,87496,12406
top,GlaxoSmithKline,INDUSTRY,SPONSOR,Sun Protection Factor Assay,Unknown,COMPLETED,Unknown,ADULT OLDER_ADULT,Healthy,TREATMENT,Unknown,Unknown,INTERVENTIONAL,PHASE1,Unknown,Unknown
freq,3374,100864,100864,14,1548,65002,809,70110,4052,66338,4828,9029,85847,22969,1176,17831


In [43]:
clin_trials['Medical Subject Headings'].value_counts()

Medical Subject Headings
Unknown                                                                                                                                                                                                  17831
Neoplasms                                                                                                                                                                                                 2197
Diabetes Mellitus Diabetes Mellitus, Type 2                                                                                                                                                               1886
Breast Neoplasms                                                                                                                                                                                          1322
Arthritis Arthritis, Rheumatoid                                                                                                                    

In [44]:
clin_trials['Start Date'] = clin_trials['Start Date'].apply(parse_mixed_date)
clin_trials['Year'] = clin_trials['Start Date'].dt.year


In [45]:
categorical_cols = ['Overall Status', 'Primary Purpose', 'Conditions', 'Study Type', 'Standard Age', 'Phases', 'Year']

In [46]:
categorical_cols

['Overall Status',
 'Primary Purpose',
 'Conditions',
 'Study Type',
 'Standard Age',
 'Phases',
 'Year']

In [47]:
clin_trials = clin_trials.drop(['Responsible Party', 'Start Date', 'Organization Class', 'Medical Subject Headings'], axis = 1)
clin_trials.head(1)

,Organization Full Name,Brief Title,Full Title,Overall Status,Standard Age,Conditions,Primary Purpose,Interventions,Intervention Description,Study Type,Phases,Outcome Measure,Year
0,TRB Chemedica International SA,Efficacy and Safety of BUFY01 Versus SVS20 in ...,Efficacy and Safety of BUFY01 Versus SVS20 in ...,NOT_YET_RECRUITING,ADULT OLDER_ADULT,Dry Eye Disease,TREATMENT,"BUFY01 eye drops in single-dose containers, SV...","1 to 2 drops in each eye, as often as needed, ...",INTERVENTIONAL,NaN,Signs,NaN


In [48]:
clin_trials['Year'].nunique()

40

In [49]:
clin_trials['Year'].isna().sum()

42688

In [50]:

encoder = OneHotEncoder(sparse=True)

# Select only categorical columns for encoding
categorical_data = clin_trials[categorical_cols]

# Encode categorical columns
encoded_trials = encoder.fit_transform(categorical_data)
encoded_df = pd.DataFrame.sparse.from_spmatrix(encoded_trials, columns=encoder.get_feature_names_out(categorical_cols))


In [51]:
clin_trials_encoded = clin_trials.drop(columns=categorical_cols)

# Combine with encoded DataFrame
clin_trials = pd.concat([clin_trials_encoded.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

In [ ]:
clin_trials.head(1)

In [ ]:
clin_trials['Organization Full Name'].value_counts()

In [ ]:
entry_count_df = clin_trials['Organization Full Name'].value_counts().reset_index()
entry_count_df.columns = ['Organization Full Name', 'entry_count']

In [ ]:
entry_count_df.head()

In [ ]:
clin_trials = clin_trials.merge(entry_count_df, on='Organization Full Name', how='left')

In [ ]:
clin_trials.head(1)

In [ ]:
combined_titles_cols = ['Brief Title', 'Full Title']
combined_inter_cols = ['Interventions', 'Intervention Description']


In [ ]:
def concat(columns):
    cols = [col for col in columns]
    clin_trials['combined_text'] = clin_trials[cols].agg(' '.join, axis=1)
    clin_trials.head(1)

In [ ]:
concat(combined_titles_cols)
clin_trials.head(1)

In [ ]:
clin_trials['combined_titles'] = clin_trials['combined_text']
clin_trials = clin_trials.drop(columns = ['combined_text'])
clin_trials.head(1)

In [ ]:
concat(combined_inter_cols)
clin_trials.head(1)

In [ ]:
clin_trials['Intervention Desc'] = clin_trials['combined_text']
clin_trials = clin_trials.drop(columns = 'combined_text')
clin_trials.head(1)

In [ ]:
clin_trials = clin_trials.drop(columns = ['Brief Title', 'Full Title','Interventions', 'Intervention Description'])
clin_trials.head(1)

In [ ]:
def preprocess_text(text):
    # Remove punctuation including '|'
    text = text.translate(str.maketrans('', '', string.punctuation + '|')).lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [ ]:
clin_trials['combined_titles'] = clin_trials['combined_titles'].apply(preprocess_text)


In [ ]:
clin_trials['Intervention Desc'] = clin_trials['Intervention Desc'].apply(preprocess_text)

In [ ]:
clin_trials['Outcome Measure'] = clin_trials['Outcome Measure'].apply(preprocess_text)

In [ ]:
vectorizer = TfidfVectorizer(min_df = .01, max_df = .9)

In [ ]:
vec_title = vectorizer.fit_transform(clin_trials['combined_titles'])

In [ ]:
vec_title_df = pd.DataFrame.sparse.from_spmatrix(vec_title, columns=vectorizer.get_feature_names_out())

In [ ]:
vec_conditions_df = pd.DataFrame.sparse.from_spmatrix(vec_conditions, columns = vectorizer.get_feature_names_out())

In [ ]:
vec_intervention = vectorizer.fit_transform(clin_trials['Intervention Desc'])

In [ ]:
vec_intervention_df = pd.DataFrame.sparse.from_spmatrix(vec_intervention, columns = vectorizer.get_feature_names_out())

In [ ]:
vec_outcome = vectorizer.fit_transform(clin_trials['Outcome Measure'])

In [ ]:
vec_outcome_df = pd.DataFrame.sparse.from_spmatrix(vec_outcome, columns = vectorizer.get_feature_names_out())

In [ ]:
clin_trials.head(1)

In [ ]:
non_sparse_cols = ['entry_count']
sparse_scaler = MaxAbsScaler()
title_sparse = sparse_scaler.fit_transform(vec_title_df)
intervention_sparse = sparse_scaler.fit_transform(vec_intervention_df)
outcome_sparse = sparse_scaler.fit_transform(vec_outcome_df)
non_sparse_scaler = StandardScaler()
scaled_non_sparse_data = non_sparse_scaler.fit_transform(clin_trials[non_sparse_cols])



In [ ]:
title_sparse_df = pd.DataFrame(title_sparse, columns=vec_title_df.columns)
intervention_sparse_df = pd.DataFrame(intervention_sparse, columns=vec_intervention_df.columns)
outcome_sparse_df = pd.DataFrame(outcome_sparse, columns=vec_outcome_df.columns)
scaled_non_sparse_df = pd.DataFrame(scaled_non_sparse_data, columns=non_sparse_cols)

In [ ]:
title_sparse_df = title_sparse_df.sparse.to_dense()
intervention_sparse_df = intervention_sparse_df.sparse.to_dense()
outcome_sparse_df = outcome_sparse_df.sparse.to_dense()
encoded_df = encoded_df.sparse.to_dense()



In [ ]:
stop

In [ ]:
print("title_sparse_df columns:", title_sparse_df.columns.tolist())
print("intervention_sparse_df columns:", intervention_sparse_df.columns.tolist())
print("outcome_sparse_df columns:", outcome_sparse_df.columns.tolist())
print("encoded_df columns:", encoded_df.columns.tolist())
print("scaled_non_sparse_df columns:", scaled_non_sparse_df.columns.tolist())

In [ ]:
concatenated_columns = (
    list(clin_trials[['Organization Full Name']].columns) +
    list(title_sparse_df.columns) +  
    list(intervention_sparse_df.columns) + 
    list(outcome_sparse_df.columns) + 
    list(encoded_df.columns) +
    list(scaled_non_sparse_df.columns) # entry_count should be here
)

In [ ]:
clin_trials = pd.concat([
    clin_trials[['Organization Full Name']].reset_index(drop=True),
    title_sparse_df.reset_index(drop=True),
    intervention_sparse_df.reset_index(drop=True),
    outcome_sparse_df.reset_index(drop=True),
    scaled_non_sparse_df.reset_index(drop=True),
    encoded_df.reset_index(drop=True),
], axis=1, ignore_index=True)
clin_trials.columns = concatenated_columns

In [ ]:
agg_dict = {
    **{col: 'mean' for col in title_sparse_df.columns},  # Mean for vectorized title features
    **{col: 'mean' for col in intervention_sparse_df.columns},  # Mean for vectorized intervention features
    **{col: 'mean' for col in outcome_sparse_df.columns},  # Mean for vectorized outcome features
    **{col: 'mean' for col in scaled_non_sparse_df.columns}, # Mean for scaled non-sparse features (entry_count)
    **{col: 'mean' for col in encoded_df.columns},  # Sum encoded categorical fields
   
}

In [ ]:
def resolve_duplicate_columns(df):
    """
    Resolve duplicate column names in a DataFrame by appending _[i] to duplicate columns.

    Parameters:
    df (pandas.DataFrame): Input DataFrame.

    Returns:
    pandas.DataFrame: DataFrame with unique column names.
    """
    seen = {}
    new_columns = []
    for column in df.columns:
        if column in seen:
            seen[column] += 1
            new_columns.append(f"{column}_{seen[column]}")
        else:
            seen[column] = 1
            new_columns.append(column)
    df.columns = new_columns
    return df

In [ ]:
clin_trials = resolve_duplicate_columns(clin_trials)

In [ ]:
clin_trials_grouped = clin_trials.groupby('Organization Full Name').agg(agg_dict)
    

In [ ]:

print(clin_trials.info())

In [ ]:
X = clin_trials.drop(columns=['Organization Full Name'])
y = clin_trials['Organization Full Name']

In [ ]:
n_components_list = [2, 5, 10, 20, 50, 100, 200]

In [ ]:
explained_variance_results = {}

for n in n_components_list:
    # Apply TruncatedSVD
    svd = TruncatedSVD(n_components=n, random_state=42)
    X_reduced = svd.fit_transform(X)

    # Calculate explained variance ratio
    explained_variance = svd.explained_variance_ratio_
    total_explained_variance = explained_variance.sum()

    # Store the results
    explained_variance_results[n] = total_explained_variance

# Print the results
for n, variance in explained_variance_results.items():
    print(f"n_components={n}: Total explained variance={variance:.4f}")

In [ ]:
final_svd = TruncatedSVD(n_components=2, random_state=42)
X = final_svd.fit_transform(X)

In [ ]:
print(len(X), len(y))

In [ ]:
clin_trials_fin = pd.concat([pd.DataFrame(X), y.reset_index(drop=True)], axis=1)

In [ ]:
clin_trials_fin.head(1)

In [ ]:
def find_closest_organizations(organization_name, n=5):
    """
    Find the n closest organization names to the given organization name.

    Parameters:
    organization_name (str): The organization name to search for.
    X_reduced_final (numpy.ndarray): 2D array containing the reduced feature vectors.
    y (pandas.Series): Series containing organization names.
    n (int): Number of closest organization names to return.

    Returns:
    list: List of the n closest organization names.
    """
    # Find the index of the organization name
    idx = y[y.str.contains(organization_name)].index[0]

    # Retrieve the corresponding reduced feature vector
    target_vector = X[idx]

    # Calculate cosine distances between the target vector and all other vectors
    distances = cosine_distances(target_vector.reshape(1, -1), X)

    # Get the indices of the n smallest distances
    closest_indices = distances.argsort(axis=1)[0][:n + 1]
    
    closest_indices = closest_indices[closest_indices != idx]
    
    # Return the closest organization names
    closest_orgs = y.iloc[closest_indices].tolist()
    return closest_orgs


In [ ]:
find_closest_organizations('Sarepta')

In [ ]:
clin_trials_fin = clin_trials_fin.rename(columns={clin_trials_fin.columns[0]: 'Component 1', clin_trials_fin.columns[1]: 'Component 2'})

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(clin_trials_fin['Component 1'], clin_trials_fin['Component 2'], c='blue', s=10, alpha=0.5)
plt.title('Reduced Data Plot')
plt.xlabel(component_names[0])
plt.ylabel(component_names[1])
plt.grid(True)
plt.show()

In [ ]:
fig = px.scatter(clin_trials_fin, x='Component 1', y='Component 2', hover_data=['Organization Full Name'])
fig.update_layout(title='Reduced Data Plot', xaxis_title='Component 0', yaxis_title='Component 1')
fig.show()